## Data export
docker-compose up

version: '3'
services:
  db:
    image: postgres:15
    environment:
      POSTGRES_USER: postgres
      POSTGRES_PASSWORD: postgres
      POSTGRES_DB: cur_learning_module
    volumes:
      # A teljes sql_files könyvtár csatolása, hogy minden .sql fájl betöltődjön
      - ./sql_files:/docker-entrypoint-initdb.d
    networks:
      - mynetwork
    ports:
      - "5432:5432"

  pgadmin:
    image: dpage/pgadmin4
    environment:
      PGADMIN_DEFAULT_EMAIL: sas@code.hu
      PGADMIN_DEFAULT_PASSWORD: adminpassword
    volumes:
      # A teljes sql_files könyvtár csatolása, hogy minden .sql fájl betöltődjön
      - ./export_files:/export
    ports:
      - "8080:80"
    depends_on:
      - db
    networks:
      - mynetwork

networks:
  mynetwork:
    driver: bridge


### DB Export scripts
#### Learning Units
select id, title, learning_units from public.cur_learning_module ;
#### Projects
select id, title, hints, learn, story, tasks, tutorial, summary from public.cur_base_project ;
#### Tutorial
select id, description from public.cur_background_material ;

## Processing dumps

In [1]:
import os
import json
import pandas as pd

In [2]:
full_learning_units = pd.read_csv("../csv/data-1727880861228.csv")
full_projects = pd.read_csv("../csv/data-1727884302987.csv")
full_tutorials = pd.read_csv("../csv/data-1727887007344.csv")

FileNotFoundError: [Errno 2] No such file or directory: '../csv/data-1727880861228.csv'

In [3]:
learning_units = full_learning_units["learning_units"]
projects = full_projects["hints"]
tutorials = full_tutorials["description"]


In [ ]:
full_learning_units

In [ ]:
learning_units

In [ ]:
projects

In [ ]:
tutorials

In [8]:
# LearningUnitAssignments frissítésére
def update_assignment(assignment, projects_df, tutorials_df):
    if assignment['type'] == 'TUTORIAL':
        # Keressük meg a tutorials_df-ben a megfelelő leírást az id alapján
        tutorial_row = tutorials_df[tutorials_df['id'] == assignment['id']]
        if not tutorial_row.empty:
            return tutorial_row.iloc[0]['description']
    elif assignment['type'] == 'PROJECT':
        # Keressük meg a projects_df-ben a megfelelő hintet az id alapján
        project_row = projects_df[projects_df['id'] == assignment['id']]
        if not project_row.empty:
            return project_row.iloc[0]['hints']
    return ''


In [11]:
# LearninUnit feldolgozása
def update_learning_units(row):
    updated_units = []
    jrow = json.loads(row)  # JSON betöltése
    for r in jrow[:5]:
        lus = r["learningUnitAssignments"]
        updated_assignments = []
        for lu in lus:
            # Frissítjük a learningUnitAssignments bejegyzést a megfelelő értékekkel
            updated_assignments.append(update_assignment(lu, full_projects, full_tutorials))
        # Frissítjük az eredeti bejegyzést az új értékekkel
        r["learningUnitAssignments"] = updated_assignments
    return jrow 



In [ ]:
print(full_learning_units["title"][0])
json.loads(learning_units[0])

In [26]:
# minden sor feldolgozása
all= []

for k in learning_units:
    all.append(update_learning_units(k))
with open("mydata.json", "w") as final:
	json.dump(all[0], final, indent=4)



In [ ]:
print(full_learning_units["title"][0])
all[0]

In [ ]:
@retry(stop=stop_after_attempt(3), wait=wait_exponential(min=1, max=10), retry=retry_if_exception_type(OpenAIError))
    async def query_openai(self, messages, max_response_tokens=32000):
        """
        Queries the OpenAI API with the given messages, using retries for reliability.

        Parameters:
            messages (list): A list of messages to send to the OpenAI API.
            max_response_tokens (int): The maximum number of tokens for the response.

        Returns:
            str: The content of the response message.

        Raises:
            OpenAIError: If an error occurs during the API call.
        """
        from util import calculate_costs
        logging.info(f"({self.name}): Querying OpenAI...")
        # Ellenőrizd a tokenek számát és szükség esetén összefoglalás
        total_tokens = self.count_tokens(messages, self.encoding)
        if total_tokens > MAX_TOKENS:
            logging.warning(f"({self.name}): Token limit exceeded ({total_tokens} tokens). Summarizing messages.")
            # Hívj meg egy összefoglaló függvényt, hogy csökkentsd a tokenek számát
            summarized_message = await self.summarize_data_with_llm(messages)
            self.memory.reset_short_term()
            self.memory.add_to_short_term(self.system_message, LOW_PRIORITY)
            self.memory.add_to_short_term(summarized_message, priority=MEDIUM_PRIORITY)
            messages = self.memory.get_short_term_messages()
            total_tokens = self.count_tokens(messages, self.encoding)
            logging.info(f"({self.name}): Token count after summarization: {total_tokens}")

        try:
            completion = await asyncio.to_thread(
                self.client.chat.completions.create,
                messages=messages,
                model=os.getenv("MODEL"),
                temperature=float(os.getenv("TEMPERATURE")),
            )
            # calculate_costs(completion['usage'], 5, 15, 1000000)
            logging.info(f"({self.name}): OpenAI query successful.")
            return completion.choices[0].message.content
        except OpenAIError as e:
            logging.error(f"({self.name}): OpenAIError: {e}", exc_info=True)
            raise